In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")


In [2]:
# Creamos un nuevo chat
import pyperclip
import uuid
from src.db.mongo.chats_manager import create_new_chat, find_chat_by_id

# user_id = uuid.uuid4()
user_id = uuid.UUID("a447f16a-2fa4-44c7-9f71-0c833b77628e")
chat_id = create_new_chat(user_id)
pyperclip.copy(str(chat_id))
print(chat_id)

Nuevo chat creado!
Conexion finalizada
7ae0ef8b-5fec-470f-8aa2-8e01028bac38


In [3]:
# Buscamos el chat en la db
import uuid
from src.db.mongo.chats_manager import find_chat_by_id

chat_id = uuid.UUID("7ae0ef8b-5fec-470f-8aa2-8e01028bac38")
chat_document = find_chat_by_id(chat_id)

chat_document_copy = chat_document.copy()

Chat has been found
Conexion finalizada


In [20]:
# Usamos la funcion chat

from src.app.app import chat

new_user_message = "but they are different"
llm_collector, collector, res = chat(
    new_user_message=new_user_message,
    chat_document=chat_document_copy,
)


generate-request
{'text': '\nsummary:  The user, Lauther, started a conversation with the assistant and asked for a list of measurement systems from the database. The assistant provided the list and the user asked for the maximum temperature of the second measurement system in the list, then asked for the date it was measured. The assistant provided the maximum temperature and the dates it was measured, and the user pointed out that the dates were different.\nuser_intent: The user wants to know the maximum temperature and the date it was measured for the second measurement system in the list.\nslots: Measurement_System_Tag=EMED-3138.12-050, Measurement_System_Name=EMED-3138.12-050'} 


enhanced-request
{'text': '\nresponse:  The user is looking for the maximum temperature and the date it was measured for the EMED-3138.12-050 measurement system.'} 


request-type
{'text': '\ntype:  complex\nanalysis: It seems to be a detailed request because the user is giving a specific measurement sys

C:\Users\lauth\OneDrive\Desktop\sql_assistant_v3\src\utils\sql_utils.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


summary-response-sql
{'text': '\nresponse: The maximum temperature recorded for the EMED-3138.12-050 measurement system was 33.354°C on April 21st, 2023.'} 


post-process-translation
{'text': 'detected_language: English.  \nresponse: The maximum temperature recorded for the EMED-3138.12-050 measurement system was 33.354°C on April 21st, 2023.'} 




In [21]:
print(llm_collector.llm_responses[0].prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Your are a very helpfull assistant<|eot_id|><|start_header_id|>user<|end_header_id|>
I need your help with a very important task for my work. Follow carefully this instructions step by step.

First, read this current summary and prevalent slots of a conversation between a virtual assistant and a user:
<Summary>The user, Lauther, started a conversation with the assistant and asked for a list of measurement systems from the database. The assistant provided the list and the user asked for the maximum temperature of the second measurement system in the list, then asked for the date it was measured.</Summary>
<Slots>Measurement_System_Tag=EMED-3138.12-050, Measurement_System_Name=EMED-3138.12-050</Slots>

Second, read this new lines to the conversation:

<Assistant>
Hi Lauther! Welcome to our conversation. How can I assist you today?
</Assistant>
<User>Could you give me a list of measurement systems from database?</User>
<Assistant

In [22]:
# Actualizamos la db con la nueva respuesta
from src.db.mongo.chats_manager import save_to_chat

save_to_chat(chat_document_copy)

Guardando chat ...
Error durante la conexion: cannot encode object: datetime.date(2023, 8, 10), of type: <class 'datetime.date'>
Conexion finalizada


In [15]:
from src.components.memory.memory import Memory

m = Memory(chat_document_copy)

print(m.list_chat_messages())


<Human>: 2024-08-05 17:25:56.212812
Hi there
<Assistant>: 2024-08-05 17:26:08.603463
Hi! Welcome to our conversation. How can I assist you today?
<Human>: 2024-08-05 17:26:31.856870
My name is Lauther
<Assistant>: 2024-08-05 17:26:41.313282
Hi Lauther! Welcome to our conversation. How can I assist you today?
<Human>: 2024-08-05 17:27:07.285992
Could you give me a list of measurement systems from database?
<Assistant>: 2024-08-05 17:28:35.193663
The database query retrieved a list of measurement systems from the "med_sistema_medicion" table, filtering results to only show active systems (Estado = 'ACT').
Here is a dataframe from SQL: 
|    | Measurement_System_Name   | Measurement_System_Tag   |
|---:|:--------------------------|:-------------------------|
|  0 | EMED-3138.11-128          | EMED-3138.11-128         |
|  1 | EMED-3138.12-050          | EMED-3138.12-050         |
|  2 | EMED-3138.12-052          | EMED-3138.12-052         |
|  3 | EMED-3138.12-065          | EMED-3138.12

In [65]:
import pandas as pd
from src.components.memory.memory_interfaces import AIMessage, HumanMessage
from typing import List

summary_instruction_no_dictionary: str = """I need your help with a very important task for my work. Follow carefully this instructions step by step.

First, read this current summary and prevalent slots of a conversation between a virtual assistant and a user:
<Summary>{current_summary}</Summary>
<Slots>{current_slots}</Slots>

Second, read this new lines to the conversation:
{chat_history}

Third, summarization. Summarize step by step the new lines and add this new summary to the previous summary.

Fourth, user intent. Find the exactly current user intention at this particular point of the conversation.

Fifth, slots. Find the prevalent slots that are necessary to the current intention.

Finally, evaluation:
- The summary should describe step by step the conversation. A correct summary should contains the slots mentioned in conversation and answer the question: "What does user and assistant talking about?".
- The user intent should contain the relevant slots he mention along the conversation. A correct intent should answer the question: "What does the user want at this point in the conversation?". 

Note that it is possible that not all conversation history is relevant and you need to summarise based on what is relevant. If the user does not have a goal at this point the intent just aim on what is the user doing: “The user is ...”.

Output format response:
The output should be formatted with the key format below. Do not add any comment before or after the key format. Just this format please.
Start Key format:
"summary" is the key and its content: Detailed summary of the current conversation in one line, do not make break lines.
"user_intent" is the key and its content: Detailed current user goal at this point of the conversation.
"slots" is the key and its content: Comma separated dictionary with slots and its values identified at this point of the conversation in one line, do not make break lines.
End of Key format

Begin!"""
summary_instruction_suffix = """summary: """

def get_new_summary_instruction(
    messages: List[AIMessage | HumanMessage] = [],
    current_summary: str | None = None,
    current_slots: str | None = None,
):
    current_summary = (
        "The next is a conversation between Assistant and Human."
        if current_summary is None
        else current_summary
    )
    messages_sorted = sorted(messages, key=lambda x: x.date_created)
    
    new_lines = ""
    for m in messages_sorted[-6:]:
        if isinstance(m, HumanMessage):
            new_lines += f"""\n<User>{m.message}</User>"""
        elif isinstance(m, AIMessage):
            if m.dataframe is not None:
                new_lines += f"""\n<Assistant>\n{m.message}\nHere is a dataframe: \n{pd.DataFrame(m.dataframe).head(10)}"""
            else:
                new_lines += f"""\n<Assistant>\n{m.message}\n</Assistant>"""

    instruction = summary_instruction_no_dictionary.format(
        current_summary=current_summary,
        current_slots=current_slots,
        chat_history=new_lines,
    )

    suffix = summary_instruction_suffix
    return instruction, suffix

In [68]:
from typing import List

messages: List[AIMessage | HumanMessage] = []

u = HumanMessage(
    message="hi there",
    message_type="Human",
)
messages.append(u)

ai = AIMessage(
    replied_user_message_id=u.message_id,
    message="Hi there, my name is M-bot",
    message_type="AI",
)
messages.append(ai)

u = HumanMessage(
    message="my name is Lauther",
    message_type="Human",
)
messages.append(u)

ai = AIMessage(
    replied_user_message_id=u.message_id,
    message="Nice to meet you Lauther, how can I help ypu?",
    message_type="AI",
)
messages.append(ai)

u = HumanMessage(
    message="i would like  alist of measurement systems from database",
    message_type="Human",
)
messages.append(u)

# ai = AIMessage(
#     replied_user_message_id=u.message_id,
#     message="Sure, here is a list of measurement systems",
#     message_type="AI",
# )
# messages.append(ai)

# u = HumanMessage(
#     message="Now give me the max temperature for the second one",
#     message_type="Human",
# )
# messages.append(u)

# ai = AIMessage(
#     replied_user_message_id=u.message_id,
#     message="Sure, the maximun temperature for EMED-050 is 35 celcius degrees",
#     message_type="AI",
# )
# messages.append(ai)

# u = HumanMessage(
#     message="Could you tell me the date when it was measured?",
#     message_type="Human",
# )
# messages.append(u)

# ai = AIMessage(
#     replied_user_message_id=u.message_id,
#     message="The maximum temperature of 31.5505 degrees Celsius was recorded on December 14, 2022, according to the data from the measurement system with the tag 'EMED-3138.12-050'.",
#     message_type="AI",
# )
# messages.append(ai)

instruction, suffix = get_new_summary_instruction(messages, "", "")